In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn import preprocessing
from sklearn import metrics
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
import re

df = pd.read_csv('Caschool.csv', encoding='latin-1')
df['const'] = 1
df= df.drop('readscr', axis = 1)
df= df.drop('mathscr', axis = 1)

df= df.drop('Unnamed: 0', axis = 1)
df= df.drop('distcod', axis = 1)
df= df.drop('county', axis = 1)
df= df.drop('district', axis = 1)
# df= df.drop('grspan', axis = 1)

print(df.head())
df = pd.get_dummies(df)
labels = np.array(df['testscr'])
df= df.drop('testscr', axis = 1)

df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size = 0.25, shuffle=False)

  grspan  enrltot   teachers    calwpct    mealpct  computer     testscr  \
0  KK-08      195  10.900000   0.510200   2.040800        67  690.799988   
1  KK-08      240  11.150000  15.416700  47.916698       101  661.200012   
2  KK-08     1550  82.900002  55.032299  76.322601       169  643.599976   
3  KK-08      243  14.000000  36.475399  77.049202        85  647.700012   
4  KK-08     1335  71.500000  33.108601  78.427002       171  640.849976   

    compstu      expnstu        str     avginc      elpct  const  
0  0.343590  6384.911133  17.889910  22.690001   0.000000      1  
1  0.420833  5099.380859  21.524664   9.824000   4.583333      1  
2  0.109032  5501.954590  18.697226   8.978000  30.000002      1  
3  0.349794  7101.831055  17.357143   8.978000   0.000000      1  
4  0.128090  5235.987793  18.671329   9.080333  13.857677      1  


In [20]:
ols_reg = LinearRegression()
ols_reg.fit(X_train, y_train)
ols_pred = ols_reg.predict(X_test)

print(metrics.mean_absolute_error(y_test, ols_pred))
print(metrics.mean_squared_error(y_test, ols_pred))
print(np.sqrt(metrics.mean_squared_error(y_test, ols_pred)))
print(np.mean(np.abs((y_test - ols_pred) / y_test)) * 100)
print(np.max(np.abs((y_test - ols_pred) / y_test)) * 100)
print(np.mean(np.abs((y_test - ols_pred))))
print(np.max(np.abs((y_test - ols_pred))))

11.111227695870392
169.64023832368784
13.024601273117263
1.6331532088206098
4.189174003351105
11.111227695870392
29.151506372043855


In [21]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'test',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    'max_depth': 8,
    'num_leaves': 128,  
    'max_bin': 512,
    'num_iterations': 100000,
    'n_estimators': 1000
}


print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)


lgbm_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print(metrics.mean_absolute_error(y_test, lgbm_pred))
print(metrics.mean_squared_error(y_test, lgbm_pred))
print(np.sqrt(metrics.mean_squared_error(y_test, lgbm_pred)))
print(np.mean(np.abs((y_test - lgbm_pred) / y_test)) * 100)
print(np.max(np.abs((y_test - lgbm_pred) / y_test)) * 100)
print(np.mean(np.abs((y_test - lgbm_pred))))
print(np.max(np.abs((y_test - lgbm_pred))))

Starting training...
[1]	valid_0's l1: 31.4901	valid_0's l2: 1097.98
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 31.4203	valid_0's l2: 1093.42
[3]	valid_0's l1: 31.3509	valid_0's l2: 1088.88
[4]	valid_0's l1: 31.2818	valid_0's l2: 1084.38
[5]	valid_0's l1: 31.213	valid_0's l2: 1079.91
[6]	valid_0's l1: 31.1446	valid_0's l2: 1075.48
[7]	valid_0's l1: 31.0753	valid_0's l2: 1070.96
[8]	valid_0's l1: 31.0076	valid_0's l2: 1066.59
[9]	valid_0's l1: 30.9402	valid_0's l2: 1062.25
[10]	valid_0's l1: 30.8731	valid_0's l2: 1057.94
[11]	valid_0's l1: 30.8082	valid_0's l2: 1053.77
[12]	valid_0's l1: 30.7436	valid_0's l2: 1049.63
[13]	valid_0's l1: 30.6811	valid_0's l2: 1045.62
[14]	valid_0's l1: 30.6171	valid_0's l2: 1041.54
[15]	valid_0's l1: 30.5549	valid_0's l2: 1037.51
[16]	valid_0's l1: 30.4933	valid_0's l2: 1033.59
[17]	valid_0's l1: 30.4271	valid_0's l2: 1029.38
[18]	valid_0's l1: 30.3644	valid_0's l2: 1025.4
[19]	valid_0's l1: 30.3019	valid_0's l2: 1021.46

[175]	valid_0's l1: 23.6951	valid_0's l2: 650.034
[176]	valid_0's l1: 23.6625	valid_0's l2: 648.419
[177]	valid_0's l1: 23.63	valid_0's l2: 646.815
[178]	valid_0's l1: 23.5977	valid_0's l2: 645.221
[179]	valid_0's l1: 23.5656	valid_0's l2: 643.637
[180]	valid_0's l1: 23.5335	valid_0's l2: 642.056
[181]	valid_0's l1: 23.5033	valid_0's l2: 640.567
[182]	valid_0's l1: 23.4731	valid_0's l2: 639.079
[183]	valid_0's l1: 23.4432	valid_0's l2: 637.609
[184]	valid_0's l1: 23.4133	valid_0's l2: 636.141
[185]	valid_0's l1: 23.3837	valid_0's l2: 634.689
[186]	valid_0's l1: 23.3545	valid_0's l2: 633.277
[187]	valid_0's l1: 23.3252	valid_0's l2: 631.843
[188]	valid_0's l1: 23.2959	valid_0's l2: 630.411
[189]	valid_0's l1: 23.2672	valid_0's l2: 629.031
[190]	valid_0's l1: 23.2393	valid_0's l2: 627.694
[191]	valid_0's l1: 23.2086	valid_0's l2: 626.219
[192]	valid_0's l1: 23.177	valid_0's l2: 624.695
[193]	valid_0's l1: 23.1454	valid_0's l2: 623.178
[194]	valid_0's l1: 23.1141	valid_0's l2: 621.675
[19

C:\Users\Orsan\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[207]	valid_0's l1: 22.7438	valid_0's l2: 604.062
[208]	valid_0's l1: 22.7175	valid_0's l2: 602.829
[209]	valid_0's l1: 22.6914	valid_0's l2: 601.603
[210]	valid_0's l1: 22.6663	valid_0's l2: 600.413
[211]	valid_0's l1: 22.6399	valid_0's l2: 599.163
[212]	valid_0's l1: 22.6135	valid_0's l2: 597.92
[213]	valid_0's l1: 22.5868	valid_0's l2: 596.658
[214]	valid_0's l1: 22.5609	valid_0's l2: 595.433
[215]	valid_0's l1: 22.5349	valid_0's l2: 594.214
[216]	valid_0's l1: 22.5086	valid_0's l2: 592.975
[217]	valid_0's l1: 22.4829	valid_0's l2: 591.772
[218]	valid_0's l1: 22.4573	valid_0's l2: 590.576
[219]	valid_0's l1: 22.4313	valid_0's l2: 589.36
[220]	valid_0's l1: 22.4033	valid_0's l2: 588.074
[221]	valid_0's l1: 22.3778	valid_0's l2: 586.878
[222]	valid_0's l1: 22.3523	valid_0's l2: 585.69
[223]	valid_0's l1: 22.326	valid_0's l2: 584.457
[224]	valid_0's l1: 22.3008	valid_0's l2: 583.282
[225]	valid_0's l1: 22.2756	valid_0's l2: 582.105
[226]	valid_0's l1: 22.2504	valid_0's l2: 580.935
[22

[404]	valid_0's l1: 19.2639	valid_0's l2: 452.833
[405]	valid_0's l1: 19.2485	valid_0's l2: 452.169
[406]	valid_0's l1: 19.2348	valid_0's l2: 451.571
[407]	valid_0's l1: 19.2234	valid_0's l2: 451.128
[408]	valid_0's l1: 19.2121	valid_0's l2: 450.687
[409]	valid_0's l1: 19.2008	valid_0's l2: 450.25
[410]	valid_0's l1: 19.1872	valid_0's l2: 449.664
[411]	valid_0's l1: 19.1766	valid_0's l2: 449.245
[412]	valid_0's l1: 19.1639	valid_0's l2: 448.736
[413]	valid_0's l1: 19.1533	valid_0's l2: 448.321
[414]	valid_0's l1: 19.1437	valid_0's l2: 447.883
[415]	valid_0's l1: 19.133	valid_0's l2: 447.428
[416]	valid_0's l1: 19.1205	valid_0's l2: 446.932
[417]	valid_0's l1: 19.108	valid_0's l2: 446.44
[418]	valid_0's l1: 19.0956	valid_0's l2: 445.95
[419]	valid_0's l1: 19.0833	valid_0's l2: 445.463
[420]	valid_0's l1: 19.0711	valid_0's l2: 444.979
[421]	valid_0's l1: 19.061	valid_0's l2: 444.576
[422]	valid_0's l1: 19.0497	valid_0's l2: 444.113
[423]	valid_0's l1: 19.0387	valid_0's l2: 443.7
[424]	va

17.731560394626158
394.7527204848904
19.868384949081552
2.598378639694417
6.149764877346253
17.731560394626158
43.46346327064464


In [22]:
eps = 5
svr = LinearSVR(epsilon=eps, C=0.01, fit_intercept=True)

svr.fit(X_train, y_train)
svr_pred = svr.predict(X_test)

print(metrics.mean_absolute_error(y_test, svr_pred))
print(metrics.mean_squared_error(y_test, svr_pred))
print(np.sqrt(metrics.mean_squared_error(y_test, svr_pred)))
print(np.mean(np.abs((y_test - svr_pred) / y_test)) * 100)
print(np.max(np.abs((y_test - svr_pred) / y_test)) * 100)
print(np.mean(np.abs((y_test - svr_pred))))
print(np.max(np.abs((y_test - svr_pred))))

68.95065292721114
8327.975137321242
91.257740150199
10.146256857063692
39.122458307193995
68.95065292721114
265.4067667073857


C:\Users\Orsan\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [23]:
eps = 3
svr = LinearSVR(epsilon=eps, C=1, fit_intercept=True)

svr.fit(X_train, y_train)
svr_pred = svr.predict(X_test)

print(metrics.mean_absolute_error(y_test, svr_pred))
print(metrics.mean_squared_error(y_test, svr_pred))
print(np.sqrt(metrics.mean_squared_error(y_test, svr_pred)))
print(np.mean(np.abs((y_test - svr_pred) / y_test)) * 100)
print(np.max(np.abs((y_test - svr_pred) / y_test)) * 100)
print(np.mean(np.abs((y_test - svr_pred))))
print(np.max(np.abs((y_test - svr_pred))))

77.54070782631642
11687.118683703105
108.10697796027371
11.375908230926365
45.69124440426916
77.54070782631642
309.96941319365123


C:\Users\Orsan\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [24]:
eps = 3
svr = LinearSVR(epsilon=eps, C=0.5, fit_intercept=True)

svr.fit(X_train, y_train)
svr_pred = svr.predict(X_test)

print(metrics.mean_absolute_error(y_test, svr_pred))
print(metrics.mean_squared_error(y_test, svr_pred))
print(np.sqrt(metrics.mean_squared_error(y_test, svr_pred)))
print(np.mean(np.abs((y_test - svr_pred) / y_test)) * 100)
print(np.max(np.abs((y_test - svr_pred) / y_test)) * 100)
print(np.mean(np.abs((y_test - svr_pred))))
print(np.max(np.abs((y_test - svr_pred))))

107.27177442557127
21013.921181895304
144.96179214501765
15.740422599714481
58.1820797077823
107.27177442557127
394.70724294220474


C:\Users\Orsan\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [25]:
svr=SVR(kernel='linear', epsilon=eps, C=10)

svr.fit(X_train, y_train)
svr_pred=svr.predict(X_test)

print(metrics.mean_absolute_error(y_test, svr_pred))
print(metrics.mean_squared_error(y_test, svr_pred))
print(np.sqrt(metrics.mean_squared_error(y_test, svr_pred)))
print(np.mean(np.abs((y_test - svr_pred) / y_test)) * 100)
print(np.max(np.abs((y_test - svr_pred) / y_test)) * 100)
print(np.mean(np.abs((y_test - svr_pred))))
print(np.max(np.abs((y_test - svr_pred))))

41.64976518037035
2970.5456640576135
54.50271244679125
6.097263313016156
20.43757089940021
41.64976518037035
141.82652575120835


In [26]:
svr=SVR(kernel='rbf', epsilon=0.5, degree=3)

svr.fit(X_train, y_train)
svr_pred=svr.predict(X_test)

print(metrics.mean_absolute_error(y_test, svr_pred))
print(metrics.mean_squared_error(y_test, svr_pred))
print(np.sqrt(metrics.mean_squared_error(y_test, svr_pred)))
print(np.mean(np.abs((y_test - svr_pred) / y_test)) * 100)
print(np.max(np.abs((y_test - svr_pred) / y_test)) * 100)
print(np.mean(np.abs((y_test - svr_pred))))
print(np.max(np.abs((y_test - svr_pred))))

29.944999171785682
1003.8667690861441
31.68385660058043
4.395861245442186
8.52427224269123
29.944999171785682
60.24529407522027
